### Allen Cell Types Database

- contains electrophysiological and morphological data for individual neurons in the mouse primary visual cortex. 

Click here for the overview: [Allen Cell Types DB](http://celltypes.brain-map.org/?_ga=2.266935630.177989909.1519313462-1702524728.1518788503)

Click here for documentation: [Allen Cell Types Documentations](http://help.brain-map.org/display/celltypes/Documentation?_ga=2.69802960.177989909.1519313462-1702524728.1518788503)

As many high-quality databases Allen Brain Atlas has API (Application Programming Interface) key. It helps us to download data from the Allen Brain programmitically, using python, for example. Allen Brain Atlas has SDK (software development kit) for python, which makes our work with the Allen Brain data in python even easier.

Allen Brain Atlas is a very user-friendly and well-documented. This tutorial is based on examples from [Allen Brain website](http://www.brain-map.org) and tutorial from [Allen Brain Github page](http://alleninstitute.github.io)

First, let's install Allen Brain Atlas SDK.

In [ ]:
import sys
!{sys.executable} -m pip install allensdk

Now, let's just see how we can download biophysical data from Allen Brain Atlas using allensdk.api.

In [ ]:
from allensdk.api.queries.cell_types_api import CellTypesApi

ct = CellTypesApi()

# a list of dictionaries containing metadata for cells with reconstructions
cells = ct.list_cells(require_reconstruction=True)

# download the electrophysiology data for one cell
ct.save_ephys_data(cells[0]['id'], 'example.nwb')

# download the reconstruction for the same cell
ct.save_reconstruction(cells[0]['id'], 'example.swc')

Let's download electrophysiology data by cell id from Allen Brain Cell Types database using allensdk.

In [ ]:
from allensdk.core.cell_types_cache import CellTypesCache

# Instantiate the CellTypesCache instance.  The manifest_file argument
# tells it where to store the manifest, which is a JSON file that tracks
# file paths.  If you supply a relative path (like this), it will go
# into your current working directory
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')

# this saves the NWB file to 'cell_types/specimen_464212183/ephys.nwb'
cell_specimen_id = 464212183
data_set = ctc.get_ephys_data(cell_specimen_id)

data_set has information on injected current stimulus waveform and the voltage response waveform for all experimental sweeps


Now we will plot one of sweeps

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

sweep_number = 30
sweep_data = data_set.get_sweep(sweep_number)

index_range = sweep_data["index_range"]
i = sweep_data["stimulus"][0:index_range[1]+1] # in A
v = sweep_data["response"][0:index_range[1]+1] # in V
i *= 1e12 # to pA
v *= 1e3 # to mV

sampling_rate = sweep_data["sampling_rate"] # in Hz
t = np.arange(0, len(v)) * (1.0 / sampling_rate)

plt.style.use('ggplot')
fig, axes = plt.subplots(2, 1, sharex=True)
axes[0].plot(t, v, color='black')
axes[1].plot(t, i, color='gray')
axes[0].set_ylabel("mV")
axes[1].set_ylabel("pA")
axes[1].set_xlabel("seconds")
plt.show()

### Metadata

Allen Brain Database store a lot of metadata.

Now we will use information from metadata to explore what is available in the database

In [ ]:
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.core.cell_types_cache import ReporterStatus as RS

# download all cells
cells = ctc.get_cells()
print("Total cells: %d" % len(cells))

# mouse cells
cells = ctc.get_cells(species=[CellTypesApi.MOUSE])
print("Mouse cells: %d" % len(cells))

# human cells
cells = ctc.get_cells(species=[CellTypesApi.HUMAN])
print("Human cells: %d" % len(cells))

# download metadata for all cells with morphology images
cells = ctc.get_cells(require_morphology = True)
print("Cells with morphology images: %d" % len(cells))

# cells with reconstructions
cells = ctc.get_cells(require_reconstruction = True)
print("Cells with reconstructions: %d" % len(cells))

# all cre positive cells
cells = ctc.get_cells(reporter_status = RS.POSITIVE)
print("Cre-positive cells: %d" % len(cells))

# cre negative cells with reconstructions
cells = ctc.get_cells(require_reconstruction = True, 
                      reporter_status = RS.NEGATIVE)
print("Cre-negative cells with reconstructions: %d" % len(cells))

About Cre-positive and Cre-negative cells: https://www.nature.com/articles/nn.2467

### Cell Morphology Reconstructions

We can download 3D reconstructions of neuronal morphologies from the Cell Types Database of Allen Brain in SWC format. Let's download one cell's reconstrution.

The 'type' i output is the type of neuronal compartment. 1 - soma, 2 - axon, 3 - dendrites, and 4 - apical dendrites (if present).
More details: morphologies also come with marker files, which contains points of interest in the reconstruction. The marker file contains locations where dendrites have been truncated due to slicing and when axons were not reconstructed. The name field indicates the type of marker (10 for dendrite truncation, 20 for no reconstruction).

In [ ]:
import pprint

# download and open an SWC file
cell_id = 464212183 #480114344
morphology = ctc.get_reconstruction(cell_id) 

# the compartment list has all of the nodes in the file
pprint.pprint(morphology.compartment_list[0])

Let's have a look at the other reconstructions

In [ ]:
# download and open an SWC file
cell_id = 480114344
morphology = ctc.get_reconstruction(cell_id) 

# the compartment list has all of the nodes in the file
pprint.pprint(morphology.compartment_list[0])

Can you tell us the differences in these two reconstructions?

Now we will download and open a marker file

In [ ]:
markers = ctc.get_reconstruction_markers(cell_id) 
pprint.pprint(markers[0])

We can use this data to draw the cell's morphology

In [ ]:
from allensdk.core.swc import Marker
fig, axes = plt.subplots(1, 2, sharey=True, sharex=True)
axes[0].set_aspect('equal', 'box-forced')
axes[1].set_aspect('equal', 'box-forced')

# Make a line drawing of x-y and y-z views
for n in morphology.compartment_list:
    for c in morphology.children_of(n):
        axes[0].plot([n['x'], c['x']], [n['y'], c['y']], color='black')
        axes[1].plot([n['z'], c['z']], [n['y'], c['y']], color='black')

# cut dendrite markers
dm = [ m for m in markers if m['name'] == Marker.CUT_DENDRITE ]

axes[0].scatter([m['x'] for m in dm], [m['y'] for m in dm], color='#3333ff')
axes[1].scatter([m['z'] for m in dm], [m['y'] for m in dm], color='#3333ff')

# no reconstruction markers
nm = [ m for m in markers if m['name'] == Marker.NO_RECONSTRUCTION ]

axes[0].scatter([m['x'] for m in nm], [m['y'] for m in nm], color='#333333')
axes[1].scatter([m['z'] for m in nm], [m['y'] for m in nm], color='#333333')

axes[0].set_ylabel('y')
axes[0].set_xlabel('x')
axes[1].set_xlabel('z')
plt.show()

### Electrophysiology Features

In [ ]:
import pandas as pd

# download all electrophysiology features for all cells
ephys_features = ctc.get_ephys_features()
ef_df = pd.DataFrame(ephys_features)

print("Ephys. features available for %d cells" % len(ef_df))

# filter down to a specific cell
specimen_id = 464212183
cell_ephys_features = ef_df[ef_df['specimen_id']== specimen_id]
cell_ephys_features

Now let's download a particular feature for all cells

In [ ]:
plt.figure()
plt.scatter(ef_df['fast_trough_v_long_square'], 
            ef_df['upstroke_downstroke_ratio_long_square'], color='#2ca25f')
plt.ylabel("upstroke-downstroke ratio")
plt.xlabel("fast trough depth (mV)")
plt.show()

What can you say about this data? Can you see two clusters here? What are the possible interpretations of these clusters?

We can check are there the excitatory (spiny) and inhibitory (aspiny) cells.

In [ ]:
ef_df.head()


In [ ]:
cells

In [ ]:
cells = ctc.get_cells()

# we want to add dendrite type as a column to the ephys. features dataframe
# first build an index on cell specimen ID, then create array of dendrite types
cell_index = { c['id']: c for c in cells }
dendrite_types = [ cell_index[cid]['dendrite_type'] for cid in ef_df['specimen_id'] ]

# now add the new column
ef_df['dendrite_type'] = pd.Series(dendrite_types, index=ef_df.index)

fig = plt.figure()

for d_type, color in [ ["spiny", "#d95f02"], ["aspiny", "#7570b3"] ]:
    df = ef_df[ef_df['dendrite_type'] == d_type]
    plt.scatter(df['fast_trough_v_long_square'], 
                df['upstroke_downstroke_ratio_long_square'], 
                color=color, label=d_type)

    plt.ylabel("upstroke-downstroke ratio")
    plt.xlabel("fast trough depth (mV)")
    plt.legend(loc='best')
    
plt.show()

### Morphology Features

We can obtain morphology features in python. 

In [ ]:
import pandas as pd

# download all morphology features for cells with reconstructions
morphology_features = ctc.get_morphology_features()

# or download both morphology and ephys features
all_features = ctc.get_all_features(require_reconstruction=True)

# convert to a pandas DataFrame
all_features = pd.DataFrame(all_features)
print("All features available for %d cells" % len(all_features))

all_features.head()

### Computing Electrophysiology Features

Allen Brain SDK can help us to compute features, not only dowload precomputed

In [ ]:
from allensdk.ephys.ephys_extractor import EphysSweepFeatureExtractor

sweep_number = 35
sweep_data = data_set.get_sweep(sweep_number)

index_range = sweep_data["index_range"]
i = sweep_data["stimulus"][0:index_range[1]+1] # in A
v = sweep_data["response"][0:index_range[1]+1] # in V
i *= 1e12 # to pA
v *= 1e3 # to mV

sampling_rate = sweep_data["sampling_rate"] # in Hz
t = np.arange(0, len(v)) * (1.0 / sampling_rate)

sweep_ext = EphysSweepFeatureExtractor(t=t, v=v, i=i, start=1.02, end=2.02)
sweep_ext.process_spikes()

print("Avg spike threshold: %.01f mV" % sweep_ext.spike_feature("threshold_v").mean())
print("Avg spike width: %.02f ms" %  (1e3 * np.nanmean(sweep_ext.spike_feature("width"))))

spike_feature() returns a NumPy array of features for each spike. You pass it the name of the feature that you want. Features that can't be calculated for a given spike are set to NaN.

In [ ]:
sweep_ext.spike_feature_keys()

Let's see when the spike occur

In [ ]:
spike_times = sweep_ext.spike_feature("threshold_t")

print(spike_times[:5])  # print just the first 5 spike times

In [ ]:
fig = plt.figure()
p = plt.plot(t, v, color='black')

min_v = v.min()

v_level = min_v - 5

plt.scatter(spike_times, np.ones(len(spike_times)) * min_v, c='firebrick')
plt.xlim(0.9, 1.2)

We can also study threshold voltages from here:

In [ ]:
fig = plt.figure()
plt.plot(t, v, color='black')

threshold_v = sweep_ext.spike_feature("threshold_v")

# setting zorder puts the dots on top of the trace
plt.scatter(spike_times, threshold_v, s=50, c='firebrick', zorder=20)
plt.xlim(1.015, 1.08)

For more fun and examples you can visit http://alleninstitute.github.io/AllenSDK/examples.html 

Help page can be found [here](http://help.brain-map.org/display/celltypes/Allen+Cell+Types+Database?_ga=2.23043128.269899930.1519587357-1702524728.1518788503)

In case you would like to store the downloaded files from Allen Brain Atlas in your collab:

In [ ]:
# First execute this line to upgrade pip
! pip install -q --upgrade "hbp-service-client==1.0.0"

Now restart your kernel

In [ ]:
# Execute the lines below
# This will save the files in the storage
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
clients.storage.upload_file('example.nwb', collab_path + '/example.nwb', 'application/x-nwb')
clients.storage.upload_file('example.swc', collab_path + '/example.swc', 'application/x-swc')